## 1. Import Libraries

In [ ]:
# Qwen
from transformers import AutoModelForCausalLM, AutoTokenizer

# Gemma
from transformers import pipeline, AutoProcessor, AutoModelForImageTextToText, Gemma3ForConditionalGeneration

# Basic Utility Libraries
import pandas as pd
import yaml
from typing import List, Dict, Optional, Union
import requests
from PIL import Image
import io
import base64
import json
import logging
import torch

# Ensure bfloat16 is available
if not hasattr(torch, 'bfloat16'):
    torch.bfloat16 = torch.float16  # Fallback for older PyTorch versions

# Set PyTorch memory management environment variables
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def clear_gpu_memory():
    """Clear GPU memory to prevent out-of-memory errors."""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("GPU memory cleared")

def get_gpu_memory_info():
    """Get current GPU memory usage information."""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        total = torch.cuda.get_device_properties(0).total_memory / 1e9
        free = total - allocated
        return {
            "allocated": allocated,
            "reserved": reserved,
            "total": total,
            "free": free
        }
    return None

## 2. Hugging-Face Model Configurations

This section defines the model IDs for the Qwen and Gemma models, as well as the set of valid classification labels we have determined.

In [ ]:
# Model IDs
QWEN_MODEL_ID = "Qwen/Qwen3-8B"
GEMMA_MODEL_ID = "google/gemma-3-12b-it"

# Valid classification labels
VALID_LABELS = {"Valid", "Spam", "Advertisement", "Irrelevant", "No Visit"}

## 3. Model Initialization

This section contains functions for initializing the Qwen and Gemma models from Hugging-Face.

### Initializing Large Langugage Model (LLM): Qwen3-8b

> Qwen3 is the latest generation of LLMs in Qwen series, delivering extensive reasoning capabilities, instruction-following, agent capabilities, and multilingual support.

### Key features:

1. Seamless switching between **thinking mode** (**for complex logical reasoning**, math, and coding) and **non-thinking mode** (for efficient, **general-purpose dialogue**)

2. Significant improvement in **reasoning capabilities** and **common sense logical reasoning**.

3. Superior **human preference alignment**, excelling in creative writing, role-playing, multi-turn dialogues, and instruction following, delivering a more **natural, engaging, and immersive conversational experience**.

4. Expertise in agentic capabilities, enabling **precise integration with external tools** in both thinking and unthinking modes.

5. **Support of 100+ languages** and dialects with strong capabilities for multilingual instruction following and translation.


> Ben: I have bolded out features that will be useful to our usecase/problem statement. Explanations are below






### Explanation of Qwen3-8b's helpful features:

1. **Thinking & Non-Thinking Modes**:

Flexibility for Nuanced Text Analysis:

Qwen3's ability to switch between "thinking mode" (for complex logic) and "non-thinking mode" (for general dialogue) will be highly valuable.
*   For difficult cases (e.g., detecting satire, fake reviews, subtle inconsistencies), the model can engage “thinking mode” for deeper logical scrutiny.
*   For routine, straightforward validation (e.g., simple keyword/spam detection), it can quickly respond using “non-thinking mode,” ensuring both accuracy and efficiency.

---

2. **Significant enhancement in reasoning capabilities and common sense logical reasoning**:

Qwen3 is excellent at spotting patterns, contradictions, and logic errors in reviews, which is critical when filtering out misleading, fake, or non-policy-compliant content.

We can rely on advanced logic to catch fake or spam reviews that **need more than standard text matching.**

---

3. **Superior human preference alignment; natural conversations**:

This has a deeper meaning. Qwen3 can generates fluent and clear responses, but would also have a deep understanding of human sentiment and tone.

It can detect subtle emotions, sarcasm, or irony in human-reviewed texts, which helps in accurately identifying fake, untrustworthy, or misleading reviews.

Such sentiment awareness makes its moderation decisions more aligned with real human judgment and intent.

---

4. **Agentic capabilities: precise integration with external tools:**

Qwen3 is designed to operate as an intelligent agent that can parse, interpret, and utilize structured data sources such as JSON files, CSVs, or API responses in real time. This allows the model to:

Ingest reviewer metadata or business records stored in JSON/CSV formats, enabling cross-referencing of review content with user profiles or transaction data.

Execute conditional logic and data lookups based on external inputs, supporting multifaceted decision-making beyond pure text analysis.

---

5. **Support for 100+ languages and dialects:**

Ensures global review classification, for multi-lingual reviews.

### Initializing Large Langugage Model (LLM): Qwen3-8b

> No more yap, lets initialize it using Python. This is interesting though

In [ ]:
def initialize_text_model(model_id: str = QWEN_MODEL_ID):
    """
    Initialize the text-only classification model.

    Args:
        model_id: Hugging Face model ID for text classification

    Returns:
        tuple: (tokenizer, model, device)
    """
    try:
        print(f"Loading text model: {model_id}")
        tokenizer = AutoTokenizer.from_pretrained(model_id)

        # Use memory-efficient loading for Qwen
        device = "cuda" if torch.cuda.is_available() else "cpu"

        if device == "cuda":
            # Check available GPU memory
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
            print(f"GPU memory available: {gpu_memory:.2f} GB")

            # Always use CPU offloading for Qwen to prevent memory issues
            print("Using CPU offloading for Qwen to prevent memory issues")
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                device_map="auto",
                torch_dtype=torch.float16,
                low_cpu_mem_usage=True
            )
        else:
            # CPU loading
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                torch_dtype=torch.float32,
                low_cpu_mem_usage=True
            )

        print(f"Text model loaded successfully on {device}")
        return tokenizer, model, device

    except Exception as e:
        logger.error(f"Failed to initialize text model: {e}")
        raise

### Initialize Multimodal Model:  Gemma-3-12b

This function initializes the multimodal image-text-to-text model (like Gemma). It uses `AutoProcessor` and `AutoModelForImageTextToText`. Crucially, it uses `device_map="auto"` and `torch_dtype=torch.float16` to efficiently load the model across available devices and manage memory, especially important for larger models.

In [ ]:
def initialize_multimodal_model(model_id: str = GEMMA_MODEL_ID):
    """
    Initialize the multimodal classification model.

    Returns: (processor, model, device)
    """
    try:
        print(f"Loading multimodal model: {model_id} with device_map='auto'")
        processor = AutoProcessor.from_pretrained(model_id)

        model = AutoModelForImageTextToText.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype="auto",   # safer than hard-forcing float16
        )

        # --- Fix pad token issue ---
        tok = processor.tokenizer
        if tok.pad_token_id is None:
            if tok.eos_token is not None:
                tok.pad_token = tok.eos_token
            else:
                tok.add_special_tokens({"pad_token": "<pad>"})
            model.config.pad_token_id = tok.pad_token_id
        tok.padding_side = "right"

        device = "cuda" if torch.cuda.is_available() else "cpu"
        print("Multimodal model loaded successfully with device_map='auto'")
        return processor, model, device

    except Exception as e:
        logger.error(f"Failed to initialize multimodal model: {e}")
        raise


## 4. Prompt Loading

This section includes a function to load the system prompt used by the models from a YAML file.

In [ ]:
def load_system_prompt(prompt_file: str = 'sample_data/BackendPrompt.yaml'):
    """
    Load the system prompt from YAML file.

    Args:
        prompt_file: Path to YAML file containing system prompt

    Returns:
        str: System prompt text
    """
    try:
        with open(prompt_file, 'r', encoding='utf-8') as f:
            config = yaml.safe_load(f)
            base_prompt = config['system_prompt']
        print(f"Successfully loaded system prompt from {prompt_file}")
        return base_prompt
    except Exception as e:
        logger.error(f"Failed to load system prompt: {e}")
        raise

### Build Text Prompt

This function formats the input for the text-only model, combining the system prompt, review text, and profile information.

In [ ]:
def build_text_prompt(review: str, system_prompt: str) -> str:
    return f"<|system|>\n{system_prompt.strip()}\n<|user|>\n{review.strip()}\n<|assistant|>"

### Build Multimodal Prompt

This function formats the input for the multimodal model, similar to the text prompt but also including specific instructions for analyzing the associated image.

In [ ]:
def build_multimodal_prompt(review: str, system_prompt: str = None) -> str:
    # system_prompt is intentionally ignored here; it's passed as SYSTEM role elsewhere
    return (
        "An image is attached. Use BOTH the image and the text to decide.\n"
        f"Review: {review}\n"
        "Respond only with the required Reason and Label."
    )


## 6. Image Loading

This section contains a helper function to load an image from various inputs (URL, file path, bytes, or PIL Image).

In [ ]:
def load_image(image_input: Union[str, bytes, Image.Image]) -> Optional[Image.Image]:
    """
    Load and prepare image for multimodal inference.

    Args:
        image_input: Image as URL, file path, bytes, or PIL Image

    Returns:
        PIL Image or None if loading fails
    """
    try:
        if isinstance(image_input, str):
            # Handle URL or file path
            if image_input.startswith(('http://', 'https://')):
                response = requests.get(image_input, timeout=10)
                response.raise_for_status()
                return Image.open(io.BytesIO(response.content))
            else:
                # File path
                return Image.open(image_input)

        elif isinstance(image_input, bytes):
            return Image.open(io.BytesIO(image_input))

        elif isinstance(image_input, Image.Image):
            return image_input

        else:
            logger.warning(f"Unsupported image input type: {type(image_input)}")
            return None

    except Exception as e:
        logger.error(f"Failed to load image: {e}")
        return None

## 7. Classification Functions

This section provides functions to perform classification using the initialized models.

In [ ]:
def classify_with_text_model(prompt: str, tokenizer, model, device) -> tuple[str, str]:
    """
    Classify using text model (Qwen-style chat format) and handle thinking/non-thinking tags.
    """
    try:
        prompt_lower = prompt.lower()

        if "/no_think" in prompt_lower:
            prompt = prompt.replace("/no_think", "").strip()
            print("Using non-thinking mode for text classification.")
        elif "/think" in prompt_lower:
            prompt = prompt.replace("/think", "").strip()
            print("Using thinking mode for text classification.")
        else:
            print("No mode tag found, defaulting to deterministic mode.")

        # Split prompt
        if "Review:" in prompt:
            parts = prompt.split("Review:")
            system_prompt = parts[0].strip()
            review = parts[1].strip()
        else:
            system_prompt = "You are a review classification system. Classify into Valid, Spam, Advertisement, Irrelevant, or No Visit."
            review = prompt.strip()

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": review}
        ]

        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        inputs = tokenizer(text, return_tensors="pt").to(device)

        generation_params = {
            "max_new_tokens": 1024,
            "temperature": 0.0,
            "do_sample": False
        }

        with torch.no_grad():
            outputs = model.generate(**inputs, **generation_params)

        full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Remove echo of user input if present (Qwen tends to reprint it)
        cleaned_output = full_output.replace(review.strip(), "").replace(system_prompt.strip(), "").strip()

        # Extract label
        label_value = extract_label(cleaned_output)

        if "Label:" in cleaned_output:
            truncated = cleaned_output.split("Label:")[0].strip()
            generated_text = f"{truncated}\nLabel: {label_value}"
        else:
            generated_text = f"{cleaned_output}\nLabel: {label_value}"

        print(f"Qwen generated text: '{generated_text}'")
        print(f"Qwen extracted label: {label_value}")

        return generated_text, label_value

    except Exception as e:
        logger.error(f"Text classification failed: {e}")
        return "Error during classification", "Unknown"


### Classify with Multimodal Model (Gemma)

This function performs classification using the initialized Gemma multimodal model. It prepares the input in the multimodal chat format and generates a response.

In [ ]:
def classify_with_multimodal_model(prompt, image, processor, model, device, system_prompt: str) -> tuple[str, str]:
    """
    Classify using Gemma multimodal model — returns (full_text, label).
    """
    try:
        print("Preparing multimodal input.")

        messages = [
            {"role": "system", "content": system_prompt or ""},
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": prompt},
                ],
            },
        ]

        chat_prompt = processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False,
        )

        inputs = processor(
            text=[chat_prompt],
            images=[image],
            return_tensors="pt",
            padding=True,
        )
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        print(f"Input tensors moved to device: {model.device}")

        with torch.inference_mode():
            outputs = model.generate(
                **inputs,
                max_new_tokens=160,
                do_sample=False,
                eos_token_id=processor.tokenizer.eos_token_id,
                pad_token_id=processor.tokenizer.pad_token_id,
            )

        input_len = inputs["input_ids"].shape[-1]
        generation = outputs[0][input_len:]
        generated_text = processor.tokenizer.decode(generation, skip_special_tokens=True).strip()

        if not generated_text:  # fallback if stripped
            raw = processor.tokenizer.decode(generation, skip_special_tokens=False)
            generated_text = raw.replace("<pad>", "").strip()

        print(f"Gemma generated text:\n{generated_text}\n")
        label = extract_label(generated_text)
        return generated_text, label

    except Exception as e:
        logger.error(f"Gemma multimodal classification failed: {e}")
        return "Error during classification", "Unknown"


## 8. Label Extraction

This section includes a function to extract the final classification label from the text generated by the models.

In [ ]:
import re
def extract_label(text: str) -> str:
    if not text or not text.strip():
        print("Extracting label from empty text.")
        return "Unknown"

    norm = text.strip()

    label_pattern = r"^\s*label\s*[:\-–—]\s*(valid|spam|advertisement|irrelevant|no visit)\s*$"
    classification_pattern = r"\*\*Classification:\*\*\s*(Valid|Spam|Advertisement|Irrelevant|No Visit)"

    # 1. Try Classification: X line
    classification_match = re.search(classification_pattern, norm, flags=re.IGNORECASE)
    if classification_match:
        label = classification_match.group(1).capitalize()
        print(f"✅ Found label from Classification line: {label}")
        return label

    # 2. Try Label: X right after </think>
    sections = re.split(r"</think>", norm, flags=re.IGNORECASE)
    label_zone = sections[1] if len(sections) > 1 else norm
    matches = re.findall(label_pattern, label_zone, flags=re.IGNORECASE | re.MULTILINE)
    if matches:
        final_label = matches[0].capitalize()
        print(f"✅ Found label in immediate zone: {final_label}")
        return final_label

    # 3. Try fallback to label anywhere
    matches = re.findall(label_pattern, norm, flags=re.IGNORECASE | re.MULTILINE)
    if matches:
        final_label = matches[0].capitalize()
        print(f"✅ Fallback label found: {final_label}")
        return final_label

    # 4. As last resort, scan for single-word label line
    lines = [ln.strip() for ln in norm.splitlines() if ln.strip()]
    for line in lines:
        if line.lower() in [l.lower() for l in VALID_LABELS]:
            print(f"✅ Found label from single word line: {line}")
            return line.capitalize()

    print("❌ No valid label found. Returning 'Unknown'")
    return "Unknown"


## 9. Main Classification Logic

This section contains the main function to classify a single review, coordinating the loading, prompting, and classification steps, including handling both text-only and multimodal cases.

In [ ]:
def classify_single_review(
    review: str,
    image_input: Optional[Union[str, bytes, Image.Image]] = None,
    text_model_components=None,           # (tokenizer, model, device)
    multimodal_model_components=None,     # (processor, model, device)
    system_prompt: str = None
) -> str:
    """
    Classify a single review.
    Returns the classification label (str).
    """
    # --- Multimodal path ---
    if image_input is not None and multimodal_model_components:
        image = load_image(image_input)
        if image:
            print("Using Gemma for multimodal classification")
            mm_user = build_multimodal_prompt(review, system_prompt)  # 3rd arg ignored

            processor, mm_model, mm_device = multimodal_model_components
            full_text, label = classify_with_multimodal_model(
                prompt=mm_user,
                image=image,
                processor=processor,
                model=mm_model,
                device=mm_device,
                system_prompt=system_prompt,   # <-- REQUIRED: pass YAML as system role
            )
            return label
        else:
            logger.warning("Image loading failed, falling back to text-only classification")

    # --- Text-only path ---
    if text_model_components and system_prompt:
        print("Using Qwen text model for text-only classification")
        txt_user = build_text_prompt(review, system_prompt)

        tokenizer, txt_model, txt_device = text_model_components
        generated_text, label = classify_with_text_model(txt_user, tokenizer, txt_model, txt_device)
        return label

    logger.warning("No model components available, returning default")
    return "Unknown"

## 10. Example Usage

This section demonstrates how to use the `classify_single_review` function with example reviews and images.

In [ ]:
def run_example_classification():
    """
    Example usage of the classification pipeline for a single review.
    """
    print("🚀 Initializing Google Review Classification Pipeline...")

    # Clear GPU memory before starting
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("🧹 GPU memory cleared before model loading")

    # Load models
    print("📚 Loading models...")

    # Option 1: Load both models for full functionality
    try:
        print("🔄 Loading Qwen text model...")
        text_components = initialize_text_model(QWEN_MODEL_ID)
        print("✅ Qwen loaded successfully")
    except Exception as e:
        print(f"⚠️ Qwen failed to load: {e}")
        print("🔄 Falling back to Gemma-only mode...")
        text_components = None

        # Clear GPU memory if Qwen failed
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("🧹 GPU memory cleared after Qwen failure")

    try:
        print("🔄 Loading Gemma multimodal model...")
        multimodal_components = initialize_multimodal_model(GEMMA_MODEL_ID)
        print("✅ Gemma loaded successfully")
    except Exception as e:
        print(f"❌ Gemma failed to load: {e}")
        raise Exception("Critical: No models available!")

    # Load system prompt
    print("📝 Loading system prompt...")
    try:
        system_prompt = load_system_prompt('sample_data/BackendPrompt.yaml')
        print("✅ System prompt loaded")
    except Exception as e:
        print(f"⚠️ Using fallback system prompt due to: {e}")
        system_prompt = "You are a review classification system. Classify reviews as Valid, Spam, Advertisement, Irrelevant, or No Visit."

    # Example 1: Text-only review with thinking tag
    print("\n🔍 Example 1: Text-only review with /think tag")
    print("-" * 50)
    review_text = "Best pizza! Visit www.pizzapromo.com for discounts! /think"

    result = classify_single_review(
        review=review_text,
        image_input=None,
        text_model_components=text_components,
        multimodal_model_components=multimodal_components,
        system_prompt=system_prompt
    )

    print(f"Review: {review_text}")
    print(f"Classification: {result}")

    # Example 2: Text-only review with no_think tag
    print("\n🔍 Example 2: Text-only review")
    print("-" * 50)
    review_text_2 = "Great service, friendly staff! /think"

    result_2 = classify_single_review(
        review=review_text_2,
        image_input=None,
        text_model_components=text_components,
        multimodal_model_components=multimodal_components,
        system_prompt=system_prompt
    )

    print(f"Review: {review_text_2}")
    print(f"Classification: {result_2}")

    # Example 3: Review with image (multimodal)
    print("\n🔍 Example 3: Review with image (multimodal)")
    print("-" * 50)
    review_text_3 = "Amazing restaurant interior! The lighting is perfect and the ambiance is wonderful."
    image_path = "sample_data/pizza.png"

    result_3 = classify_single_review(
        review=review_text_3,
        image_input=image_path,
        text_model_components=text_components,
        multimodal_model_components=multimodal_components,
        system_prompt=system_prompt
    )

    print(f"Review: {review_text_3}")
    print(f"Image: {image_path}")
    print(f"Classification: {result_3}")

    # Example 4: Review with pizza image
    print("\n🔍 Example 4: Review with interior image")
    print("-" * 50)
    image_path_pizza = "sample_data/interior.png"
    review_text_4 = "Check out this amazing pizza!"

    result_4 = classify_single_review(
        review=review_text_4,
        image_input=image_path_pizza,
        text_model_components=text_components,
        multimodal_model_components=multimodal_components,
        system_prompt=system_prompt
    )

    print(f"Review: {review_text_4}")
    print(f"Image: {image_path_pizza}")
    print(f"Classification: {result_4}")

    return result, result_2, result_3, result_4

## 11. Run Example and Display Results

This cell calls the `run_example_classification` function and prints the results.

In [ ]:
try:
    results = run_example_classification()
    print("\n--- Function Return Values ---")
    print(f"Example 1 Result: {results[0]}")
    print(f"Example 2 Result: {results[1]}")
    print(f"Example 3 Result: {results[2]}")
    print(f"Example 4 Result: {results[3]}")

    print("\n--- Summary ---")
    print(f"Total examples processed: {len(results)}")
    print(f"Successful classifications: {sum(1 for r in results if r != 'Valid' or 'Error' not in str(r))}")

except Exception as e:
    print(f"❌ Pipeline execution failed: {e}")
    print("Please check the error messages above for debugging information.")

## 10.1. Example Usage with JSON input fed

In [ ]:
def classify_json_reviews(
    json_reviews,
    text_model_components=None,
    multimodal_model_components=None,
    system_prompt: str = None,
    build_text_prompt=None
):
    import torch

    # Clear GPU memory
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("🧹 GPU memory cleared")

    results = []
    total_reviews = len(json_reviews)

    print(f"📊 Processing {total_reviews} reviews...")

    for i, review in enumerate(json_reviews, 1):
        print(f"🔍 Processing review {i}/{total_reviews} (ID: {review.get('reviewId', 'Unknown')})")
        image_input = review.get("reviewImageUrls", None)

        # Handle case where image_input might be a list of URLs
        if isinstance(image_input, list) and len(image_input) > 0:
            image_input = image_input[0]  # Take the first image if multiple exist
        elif not image_input or image_input == "":
            image_input = None

        # Only include non-empty fields
        text_fields = []
        fields_to_include = [
            ("Review ID", "reviewId"),
            ("Location/Business", "title"),
            ("Review", "text"),
            ("Translated Review", "textTranslated"),
            ("Stars", "stars"),
            ("Response from Owner", "responseFromOwnerText"),
            ("Reviewer ID", "reviewerId"),
            ("Published At", "publishedAtDate"),
        ]

        for label, key in fields_to_include:
            value = review.get(key)
            if value not in [None, "", "null"]:
                text_fields.append(f"{label}: {value} /think")

        combined_text = " | ".join(text_fields)

        # ✨ Build the prompt ONLY if using Qwen (text-only) and build_text_prompt is supplied
        if image_input is None and build_text_prompt and system_prompt:
            combined_text = build_text_prompt(combined_text, system_prompt)

        # Call your existing pipeline
        try:
            result = classify_single_review(
                review=combined_text,
                image_input=image_input,
                text_model_components=text_model_components,
                multimodal_model_components=multimodal_model_components,
                system_prompt=system_prompt
            )

            results.append({
                "reviewId": review.get("reviewId"),
                "result": result
            })
            print(f"✅ Review {i} classified as: {result}")

        except Exception as e:
            print(f"⚠️ Failed to classify review {i}: {e}")
            results.append({
                "reviewId": review.get("reviewId"),
                "result": "Error",
                "error": str(e)
            })

    return results


In [ ]:
import json
import torch

# Clear GPU memory first
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("�� GPU memory cleared")

# Initialize the text model (Qwen)
print("🔄 Loading Qwen text model...")
text_components = initialize_text_model(QWEN_MODEL_ID)
print("✅ Qwen loaded successfully")

# Load the system prompt
print("📝 Loading system prompt...")
system_prompt = load_system_prompt('sample_data/BackendPrompt.yaml')
print("✅ System prompt loaded")

# Load your reviews
print("�� Loading reviews...")
with open("sample_data/ground_truth_no_labels.json", "r", encoding="utf-8") as f:
    reviews = json.load(f)
print(f"✅ Loaded {len(reviews)} reviews")

# Now run your classification function
print("🚀 Starting classification...")
results = classify_json_reviews(
    json_reviews=reviews,
    text_model_components=text_components,
    system_prompt=system_prompt,
    build_text_prompt=build_text_prompt
)

# Save results
print("💾 Saving results...")
with open("hotel_reviews_classified.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print("✅ Results saved to hotel_reviews_classified.json")

print(f"�� Completed! Processed {len(results)} reviews")

�� GPU memory cleared
🔄 Loading Qwen text model...
Loading text model: Qwen/Qwen3-8B
GPU memory available: 42.47 GB
Using CPU offloading for Qwen to prevent memory issues


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Text model loaded successfully on cuda
✅ Qwen loaded successfully
📝 Loading system prompt...
Successfully loaded system prompt from sample_data/BackendPrompt.yaml
✅ System prompt loaded
�� Loading reviews...
✅ Loaded 421 reviews
🚀 Starting classification...
🧹 GPU memory cleared
📊 Processing 421 reviews...
🔍 Processing review 1/421 (ID: ChdDSUhNMG9nS0VJQ0FnTURBMU1uU3RnRRAB)
Using Qwen text model for text-only classification
Using thinking mode for text classification.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Found label from single word line: Valid
Qwen generated text: 'system

user

assistant
<think>
Okay, let's tackle this review classification. First, I need to figure out the business category for Westlite Ubi. The name sounds like it could be a retail store or maybe a service center. Since the user hasn't provided specific details, I'll have to make an educated guess. "Ubi" might be a typo or a specific location name, but without more context, I'll assume it's a retail or service business.

Now, looking at the review itself. The review ID is a long string, which might be a placeholder or a system-generated ID. The stars are 5, which is positive. The reviewer ID is a number in scientific notation, which is unusual but not necessarily indicative of spam. The published date is in the future (2025-08-07), which is odd because the current date is 2023. That might be a red flag, but maybe it's a test or a mistake. However, the user's instruction says to focus on the review text, not the da

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✅ Found label from single word line: Valid
Qwen generated text: 'system

user

assistant
<think>
Okay, let's tackle this review. First, I need to determine the business category. The location is "Urban Hostel @ Aljunied," which is a hostel, so the category should be 'accommodation' or 'hotel' under the services category.

Now, looking at the review content. The user mentions the manager, the team's kindness, the location's proximity to MRT, staying for a month, security, comfortable beds, services working perfectly, breakfast and coffee included, affordability in Singapore. All these are relevant to a hostel's services. The user also gives a 5-star rating, which matches the positive sentiment. There's no mention of unrelated products, no spammy elements like URLs or excessive emojis, and no indication that the reviewer didn't visit. The review is detailed and specific to the hostel's offerings. So, it doesn't fit into Advertisement, Spam, Irrelevant, or No Visit. Therefore, it should b

KeyboardInterrupt: 